In [1]:
!pip install gspread
!pip install gspread-dataframe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import auth
auth.authenticate_user()
import gspread

from google.colab import auth
auth.authenticate_user()

from gspread_dataframe import get_as_dataframe, set_with_dataframe
import gspread
from google.auth import default
import pandas as pd

#obtener credenciales y acceder a la spreadsheet
creds, _ = default()
gc = gspread.authorize(creds)
wb = gc.open_by_key('1sKR0fjKL4OAo0CJ8nEcXxFgp7JpG_KcJu7nSygswqgU')
ws = wb.worksheet('RawData')


df = get_as_dataframe(ws, usecols=[0,1,2,3]) #usar las primeras 4 columnas dado que ahí están los datos,  y convertirlo a data frame
df = df.dropna() #remover todos los nan


In [4]:
#importar cuda
from numba import cuda
import numpy as np

In [27]:
#cuda
@cuda.jit
def conversor(c,f,k):
    i = cuda.grid(1)
    if i < len(c):
       f[i] =  (9/5)*c[i] + 32
       k[i] = 273.15 + c[i]

In [28]:
#crear variables
x = df.iloc[0:,0].values.astype(np.float64)
f = np.zeros_like(x)
k = np.zeros_like(x)
#cantidad de datos a procesar
N = len(x)

In [45]:
#calcular los bloques (se tiene 256 hilos por bloque)
threads = 256
blocks = (N // threads) + 1
#iniciar el kernel
conversor[blocks, threads](x,f,k)
#colocar los valores en el dataframe
df["Temperatura(°F)"] = f
df["Temperatura(K)"] = k

#cambiar de los 1.0 a True y los 0's a False
df.loc[df["Iluminacion"] >= 1, "Iluminacion"] = True
df.loc[df["Iluminacion"] < 1, "Iluminacion"] = False

print(df)

     Temperatura(°C)  Temperatura(°F)  Temperatura(K) Iluminacion
0              26.24           79.232          299.39        True
1              26.23           79.214          299.38        True
2              26.24           79.232          299.39        True
3              26.25           79.250          299.40        True
4              26.25           79.250          299.40        True
..               ...              ...             ...         ...
175            27.43           81.374          300.58        True
176            27.42           81.356          300.57        True
177            27.42           81.356          300.57        True
178            27.42           81.356          300.57        True
179            27.41           81.338          300.56        True

[180 rows x 4 columns]


/usr/local/lib/python3.7/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [46]:
#subir los valores a la datasheet
set_with_dataframe(ws, df)